In [9]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import Sequential, Model

In [5]:
df_short = pd.read_csv("data/processed/processed_short.csv")
df_medium = pd.read_csv("data/processed/processed_medium.csv")
df_dank= pd.read_csv("data/processed/processed_dank.csv")

In [7]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split



vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)

In [6]:
def create_embedding_layer(df):
    name =[x for x in globals() if globals()[x] is df][0]
    #text = df['text'].to_numpy()
    text = np.asarray(df['text'])
    y = df['label']
    vectorizer.adapt(text)

    voc = vectorizer.get_vocabulary()
    word_index = dict(zip(voc, range(len(voc))))
    print("data/" + str(name) + ".wordvectors")
    wv = KeyedVectors.load("data/" + str(name) + ".wordvectors", mmap='r')
    embeddings_index= {}
    for word in voc:
        if wv.has_index_for(word):
            embeddings_index[word] = wv[word]
            
        else:
            embeddings_index[word] = np.zeros(100)
    
    num_tokens = len(voc) + 2
    embedding_dim = 100
    hits = 0
    misses = 0
    
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            hits += 1
        else:
            misses += 1
    print("Converted %d words (%d misses)" % (hits, misses))

    embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,)

    return embedding_layer

In [13]:
def build_model(df):
    model = Sequential()
    model.add(create_embedding_layer(df))
    model.add(Bidirectional(LSTM(64, return_sequences=True)))
    model.add(Bidirectional(LSTM(64, return_sequences=False)))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])  

    print(model.summary())
    return model


In [15]:
def train_model(df):
    name =[x for x in globals() if globals()[x] is df][0]
    #creating the desired vectors
    text = np.asarray(df['text'])
    y = df["label"]
    text_train, text_test, y_train, y_test = train_test_split(
    text, y, test_size=0.33, random_state=42)

    text_train, text_val, y_train, y_val = train_test_split(
    text_train, y_train, test_size=0.33, random_state=42)

    X_train = vectorizer(text_train)
    X_val = vectorizer(text_val)
    X_test = vectorizer(text_test)

    model = build_model(df)

    model.fit([X_train, X_train, X_train],
                     y_train,
                     batch_size=64,
                     epochs=10,
                     validation_data=([X_val, X_val, X_val],
                     y_val))
    
    loss, accuracy =  model.evaluate([X_test, X_test, X_test], y_test)

    print("Loss: ", loss)
    print("Accuracy: ", accuracy)
        
    return model

In [16]:
train_model(df_dank)

data/df_dank.wordvectors
Converted 12428 words (0 misses)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1243000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 128)         84480     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               12900     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 1,439,297
Trainable params: 196,297
Non-trainabl

NameError: name 'loss' is not defined